In [1]:
import happybase
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
connection = happybase.Connection('localhost', port=9090)

table_name = b'pogoda_hist'
table = connection.table(table_name)

In [3]:
#for key, data in table.scan():
#    print(data[b"data:rodzaj_opadu"])

In [4]:
pogoda_df = []
for key, data in table.scan():
    tmp = []
    tmp.append(float(key.decode()))
    tmp.append(int(data[b"data:r"].decode()))
    tmp.append(int(data[b"data:m"].decode()))
    tmp.append(int(data[b"data:d"].decode()))
    tmp.append(float(data[b"data:tmax"].decode()))
    tmp.append(float(data[b"data:tmin"].decode()))
    tmp.append(float(data[b"data:tstd"].decode()))
    tmp.append(float(data[b"data:smdb"].decode()))
    try:
        tmp.append(data[b"data:rodzaj_opadu"].decode())
    except:
        tmp.append("nodata")
    tmp.append(float(data[b"data:pksn"].decode()))
    pogoda_df.append(tmp)

In [5]:
#pogoda_df

In [6]:
table_name = b'przejazdy_hist'
table = connection.table(table_name)

In [7]:
przejazdy_df = []
for key, data in table.scan():
    tmp = []
    tmp.append(float(key.decode()))
    tmp.append(int(data[b"data:r"].decode()))
    tmp.append(int(data[b"data:m"].decode()))
    tmp.append(int(data[b"data:d"].decode()))
    tmp.append(data[b"data:departure_time_x"].decode())
    tmp.append(data[b"data:departure_time_y"].decode())
    tmp.append(float(data[b"data:diff"].decode()))
    tmp.append(data[b"data:route_short_name"].decode())
    tmp.append(int(data[b"data:route_type"].decode()))
    przejazdy_df.append(tmp)

TypeError: __str__ returned non-string (type bytes)

In [9]:
df1 = pd.DataFrame(przejazdy_df, columns=['timestamp', 'rok', 'mc', 'd', 'odjazd_x', 'odjazd_y', 'diff', 'nazwa_trasy', 'typ'])
df1.head()

,timestamp,rok,mc,d,odjazd_x,odjazd_y,diff,nazwa_trasy,typ
0,2.022090e+15,2022,9,2,05:15:00,05:16:50,110.0,1,0
1,2.022090e+15,2022,9,2,05:17:00,05:17:10,-10.0,1,0
2,2.022090e+15,2022,9,2,05:19:00,05:20:50,90.0,1,0
3,2.022090e+15,2022,9,2,05:21:00,05:22:20,35.0,1,0
4,2.022090e+15,2022,9,2,05:22:00,05:23:20,65.0,1,0


In [10]:
df2 = pd.DataFrame(pogoda_df, columns=['date', 'rok', 'mc', 'd', 'tmax', 'tmin', 'tmean', 'opad_num', 'opad_rodz', 'snieg'])
df2.head()

,date,rok,mc,d,tmax,tmin,tmean,opad_num,opad_rodz,snieg
0,20220101.0,2022,1,1,10.7,5.6,7.6,0.3,W,0.0
1,20220102.0,2022,1,2,9.5,3.7,6.6,3.2,W,0.0
2,20220103.0,2022,1,3,9.9,6.3,7.6,4.3,W,0.0
3,20220104.0,2022,1,4,6.7,4.6,5.8,7.0,W,0.0
4,20220105.0,2022,1,5,7.3,2.7,4.5,3.8,W,0.0


In [ ]:
#Suma dobowa opadów [mm]                 8/1
#Rodzaj opadu  [S/W/ ]                   1
#Wysokość pokrywy śnieżnej [cm]          5

In [11]:
#df1.astype({'date': 'str'}).dtypes
df1['date'] = df1['timestamp']
df1['date'] = df1['date'].astype(str).str.slice(0,8)
df1.head()

,timestamp,rok,mc,d,odjazd_x,odjazd_y,diff,nazwa_trasy,typ,date
0,2.022090e+15,2022,9,2,05:15:00,05:16:50,110.0,1,0,20220902
1,2.022090e+15,2022,9,2,05:17:00,05:17:10,-10.0,1,0,20220902
2,2.022090e+15,2022,9,2,05:19:00,05:20:50,90.0,1,0,20220902
3,2.022090e+15,2022,9,2,05:21:00,05:22:20,35.0,1,0,20220902
4,2.022090e+15,2022,9,2,05:22:00,05:23:20,65.0,1,0,20220902


In [12]:
df2['date'] = df2['date'].astype(str).str.slice(0,8)
df2.head()

,date,rok,mc,d,tmax,tmin,tmean,opad_num,opad_rodz,snieg
0,20220101,2022,1,1,10.7,5.6,7.6,0.3,W,0.0
1,20220102,2022,1,2,9.5,3.7,6.6,3.2,W,0.0
2,20220103,2022,1,3,9.9,6.3,7.6,4.3,W,0.0
3,20220104,2022,1,4,6.7,4.6,5.8,7.0,W,0.0
4,20220105,2022,1,5,7.3,2.7,4.5,3.8,W,0.0


In [13]:
df_joined = df1.merge(df2.iloc[:, [0, 4, 5, 6, 7, 8, 9]], how='left', on='date')
df_joined

,timestamp,rok,mc,d,odjazd_x,odjazd_y,diff,nazwa_trasy,typ,date,tmax,tmin,tmean,opad_num,opad_rodz,snieg
0,2.022090e+15,2022,9,2,05:15:00,05:16:50,110.0,1,0,20220902,21.7,10.4,15.5,0.0,nodata,0.0
1,2.022090e+15,2022,9,2,05:17:00,05:17:10,-10.0,1,0,20220902,21.7,10.4,15.5,0.0,nodata,0.0
2,2.022090e+15,2022,9,2,05:19:00,05:20:50,90.0,1,0,20220902,21.7,10.4,15.5,0.0,nodata,0.0
3,2.022090e+15,2022,9,2,05:21:00,05:22:20,35.0,1,0,20220902,21.7,10.4,15.5,0.0,nodata,0.0
4,2.022090e+15,2022,9,2,05:22:00,05:23:20,65.0,1,0,20220902,21.7,10.4,15.5,0.0,nodata,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2733995,2.023052e+15,2023,5,23,08:44:00,08:45:05,43.0,124,3,20230523,25.2,11.3,19.4,0.0,nodata,0.0
2733996,2.023052e+15,2023,5,23,08:45:00,08:46:23,60.0,124,3,20230523,25.2,11.3,19.4,0.0,nodata,0.0
2733997,2.023052e+15,2023,5,23,08:46:00,08:47:18,68.0,124,3,20230523,25.2,11.3,19.4,0.0,nodata,0.0
2733998,2.023052e+15,2023,5,23,08:47:00,08:48:14,62.0,124,3,20230523,25.2,11.3,19.4,0.0,nodata,0.0


In [19]:
df_joined.to_csv("joined_data.csv")